In [2]:
!pip install cvzone


In [1]:
import torch
torch.__version__


'2.3.1+cu121'

In [6]:
import cv2
import supervision as sv
from ultralytics import YOLOv10
import os
import smtplib
from email.message import EmailMessage
from datetime import datetime

# Email configuration
smtp_server = 'smtp.gmail.com'
smtp_port = 587
email_sender = 'siddhantrane80@gmail.com'
email_password = 'tfxglgqvyajehttk'
email_recipient = 'ranesid3197@gmail.com'

# Function to send email
def send_email(image_path):
    try:
        msg = EmailMessage()
        msg['Subject'] = 'Fall Detected Alert'
        msg['From'] = email_sender
        msg['To'] = email_recipient
        msg.set_content('A fall has been detected. Please find the attached image.')

        with open(image_path, 'rb') as img:
            img_data = img.read()
            img_name = os.path.basename(image_path)
            msg.add_attachment(img_data, maintype='image', subtype='jpeg', filename=img_name)

        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(email_sender, email_password)
            server.send_message(msg)
            print(f"Email sent to {email_recipient} with attachment {img_name}")

    except Exception as e:
        print(f"Failed to send email: {e}")

# Load the YOLO model
model = YOLOv10('best.pt')

# Initialize annotators
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Open the webcam
cap = cv2.VideoCapture(1)  # Use the default webcam (index 1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

save_directory = "detected_falls"
os.makedirs(save_directory, exist_ok=True)

fall_detected = False  # Flag to check if fall has been detected
fall_buffer_count = 0  # Counter to create a buffer
fall_buffer_threshold = 20  # Number of frames to wait before saving

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    # Perform detection
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Annotate the frame with detections
    annotated_image = bounding_box_annotator.annotate(
        scene=frame, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)

    # Check if a fall is detected
    for detection in detections:
        label = detection[5]['class_name']  # Extract the label from the dictionary in the detection tuple
        if isinstance(label, str) and label == 'Fall-Detected':
            fall_buffer_count += 1
            if not fall_detected and fall_buffer_count >= fall_buffer_threshold:
                print("Fall detected and buffer reached. Saving the image.")
                
                # Generate a unique filename using the current timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                image_path = os.path.join(save_directory, f"fall_detected_{timestamp}.jpg")
                
                success = cv2.imwrite(image_path, annotated_image)  # Save the annotated image
                if success:
                    print(f"Image successfully saved to {image_path}")
                    send_email(image_path)  # Send the email with the saved image
                else:
                    print("Error: Failed to save the image.")
                
                fall_detected = True  # Set flag to avoid capturing again
                fall_buffer_count = 0  # Reset the buffer count
            break

    # If no fall is detected, reset the flag and buffer count
    if not any(isinstance(detection[5]['class_name'], str) and detection[5]['class_name'] == 'Fall-Detected' for detection in detections):
        fall_detected = False
        fall_buffer_count = 0  # Reset buffer count when no fall is detected

    # Display the annotated frame
    cv2.imshow('Webcam Feed', annotated_image)

    # Add a delay and check for ESC key press to close the window
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        print("Escape hit, closing...")
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 32.4ms
Speed: 4.2ms preprocess, 32.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 42.9ms
Speed: 3.5ms preprocess, 42.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 31.1ms
Speed: 2.0ms preprocess, 31.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 27.6ms
Speed: 2.0ms preprocess, 27.6ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 37.5ms
Speed: 2.2ms preprocess, 37.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 27.1ms
Speed: 3.0ms preprocess, 27.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 28.7ms
Speed: 3.7ms preprocess, 28.7ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 26.0ms
Speed: 2.1ms preprocess, 26.0ms i